## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [1]:
from datetime import datetime
from scipy import stats  
import os
import pytz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  

%matplotlib inline

### Creating IDX Equities Time Series

In [2]:
# idx_combined = pd.read_csv('/home/nikki/documents/amibroker/idx_exported_combined.csv', parse_dates=[1])

In [3]:
# Location of the IDX Stocks OHLC csv files
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv'
# csv_loc = '/Users/nikki/Documents/idx_exported_csv'
csv_loc = '/home/nikki/documents/amibroker/exported-csv/'

# Backup if the first one doesn't work
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv_gama'

tickers = os.listdir(csv_loc)

In [4]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for ticker in tickers:
    data_idx[ticker[:-4]] = pd.read_csv(f'{csv_loc}/{ticker}',
                                        index_col='date',
                                        parse_dates=['date'])

In [5]:
data_idx['ASII'].tail()

,open,high,low,close,volume
date,,,,,
2020-09-21,4790.0,4800.0,4620.0,4670.0,23388100
2020-09-22,4600.0,4740.0,4590.0,4670.0,22893300
2020-09-23,4680.0,4730.0,4590.0,4630.0,15696100
2020-09-24,4620.0,4650.0,4560.0,4600.0,24393300
2020-09-25,4610.0,4920.0,4610.0,4860.0,40064600


### Stocks on the Move


In [6]:
def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

In [7]:
def volatility(ts, period=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [8]:
# How many (series) candles back for momentum calculation?
momentum_window = 128

# Create an empty DataFrame to store score
momentum_table = pd.DataFrame(columns=['ticker', 'score', 'vola', 'inv_vola'])

# Eliminated stocks list
eliminated_stocks = pd.DataFrame(columns=['ticker', 'score', 'vola', 'reason'])

# How many (series) candles+1 back for std dev calculation?
vola_window = 24

# How many (series) candles back for EWMA calculation?
ewma_period = 128

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_idx.items():
    momentum_series = timeseries['close'].iloc[-momentum_window:]
    score = momentum_score(momentum_series)
    vola_series = timeseries['close']
    vola = volatility(vola_series, vola_window) * 16
    median_volume = timeseries['volume'].rolling(vola_window).median().iloc[-1]
    ewma = timeseries['close'].ewm(span=ewma_period).mean().iloc[-1]

    # Need the stocks to exist at least 3 years prior (756 trading days)
    if len(timeseries) < 756:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'umur belum 3 tahun'},
                                                     ignore_index=True)
        continue
    
    # If median volume falls below 100k in the stocks, drop it 
    if median_volume < 100000:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'avg volume di bawah 100k'},
                                                     ignore_index=True)
        continue
        
    # If it has been suspended (daily vol == 0) more than once, drop it
    if timeseries['volume'].iloc[-momentum_window:].tolist().count(0) > 1:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'pernah disuspend lebih dari 1x'},
                                                     ignore_index=True)
        continue
    
    momentum_table = momentum_table.append({'ticker': ticker,
                                            'score': score,
                                            'vola': vola,
                                            'median_vol': median_volume,
                                            'ewma': ewma},
                                           ignore_index=True)
    
    momentum_table['inv_vola'] = 1 / momentum_table['vola']

In [9]:
print(f'Ada {len(momentum_table)} saham lolos')
print(f'Ada {len(eliminated_stocks)} saham tereliminasi')

Ada 199 saham lolos
Ada 504 saham tereliminasi


In [10]:
momentum_table.sort_values('score', ascending=False)[:50].to_clipboard()

In [11]:
momentum_table.sort_values('score', ascending=False)[:20]

,ticker,score,vola,inv_vola,ewma,median_vol
57,PYFA,4560.870566,0.932255,1.072668,636.587522,921900.0
134,ARTO,3853.967700,0.323402,3.092131,1981.933971,143850.0
11,WIIM,2740.664951,0.755799,1.323103,247.340039,28137150.0
174,AGRO,1940.100749,0.875202,1.142593,227.673706,51448950.0
39,INAF,1361.744036,0.986801,1.013376,1983.362892,2236150.0
116,CENT,1135.019384,0.728956,1.371824,101.320833,15092800.0
29,MARK,817.569276,0.464049,2.154945,665.782950,3343150.0
32,KAEF,797.967173,0.995550,1.004470,2103.174887,10512200.0
82,KRAS,731.811091,0.524681,1.905920,308.763623,28565450.0
133,DOID,673.181175,0.532924,1.876440,222.129505,47070950.0


In [12]:
momentum_table.sort_values('score', ascending=False)[:20]

,ticker,score,vola,inv_vola,ewma,median_vol
57,PYFA,4560.870566,0.932255,1.072668,636.587522,921900.0
134,ARTO,3853.967700,0.323402,3.092131,1981.933971,143850.0
11,WIIM,2740.664951,0.755799,1.323103,247.340039,28137150.0
174,AGRO,1940.100749,0.875202,1.142593,227.673706,51448950.0
39,INAF,1361.744036,0.986801,1.013376,1983.362892,2236150.0
116,CENT,1135.019384,0.728956,1.371824,101.320833,15092800.0
29,MARK,817.569276,0.464049,2.154945,665.782950,3343150.0
32,KAEF,797.967173,0.995550,1.004470,2103.174887,10512200.0
82,KRAS,731.811091,0.524681,1.905920,308.763623,28565450.0
133,DOID,673.181175,0.532924,1.876440,222.129505,47070950.0


In [13]:
eliminated_stocks[eliminated_stocks['reason'] == 'umur belum 3 tahun'].sort_values('score', ascending=False)

,ticker,score,vola,reason
40,BRIS,3815.691907,9.371545e-01,umur belum 3 tahun
82,DMMX,1200.297792,6.199791e-01,umur belum 3 tahun
44,CBMF,601.234993,3.262737e-01,umur belum 3 tahun
171,SAPX,529.212255,3.554760e-01,umur belum 3 tahun
43,INDO,466.787826,4.488161e-01,umur belum 3 tahun
...,...,...,...,...
83,KBAG,-85.764465,1.415350e+00,umur belum 3 tahun
0,ITIC,-90.342123,4.416746e-01,umur belum 3 tahun
172,HRME,-91.511054,5.381667e-09,umur belum 3 tahun
209,ARKA,-92.140816,5.981414e-01,umur belum 3 tahun


In [14]:
eliminated_stocks[eliminated_stocks['reason'] == 'pernah disuspend lebih dari 1x'].sort_values('score', ascending=False)

,ticker,score,vola,reason
163,TIFA,779.395081,0.305628,pernah disuspend lebih dari 1x
261,INCI,272.108060,0.410409,pernah disuspend lebih dari 1x
128,RANC,100.071218,0.348120,pernah disuspend lebih dari 1x
39,RMBA,74.438505,1.595118,pernah disuspend lebih dari 1x
307,RUIS,42.147570,0.923970,pernah disuspend lebih dari 1x
488,BIPP,39.918142,0.606751,pernah disuspend lebih dari 1x
109,WINS,35.294318,0.213005,pernah disuspend lebih dari 1x
60,LPPS,15.956782,0.699902,pernah disuspend lebih dari 1x
218,GWSA,13.444831,0.528288,pernah disuspend lebih dari 1x
34,AISA,8.955749,1.638821,pernah disuspend lebih dari 1x


In [15]:
eliminated_stocks[eliminated_stocks['reason'] == 'avg volume di bawah 100k'].sort_values('score', ascending=False)

,ticker,score,vola,reason
77,ARGO,489.785755,1.123822e+00,avg volume di bawah 100k
159,PTIS,358.443206,9.486231e-01,avg volume di bawah 100k
408,BBHI,323.138029,5.734052e-01,avg volume di bawah 100k
436,MERK,306.264092,4.505298e-01,avg volume di bawah 100k
358,PNSE,165.606997,3.973270e-07,avg volume di bawah 100k
...,...,...,...,...
70,PGLI,-66.517070,1.117693e+00,avg volume di bawah 100k
377,NASA,-68.554597,2.249972e-08,avg volume di bawah 100k
419,TARA,-71.126410,3.709058e-08,avg volume di bawah 100k
373,GLOB,-79.339016,5.216332e-01,avg volume di bawah 100k
